In [ ]:
import os
import shutil
import gdown
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from collections import Counter
from xml.etree import ElementTree
import cv2
import matplotlib.pyplot as plt
from IPython.display import Image, Video
from ultralytics import YOLO

In [ ]:
annot_data_path = 'train\Annotations Boxes'
train_data_path = 'train\img-train'
test_data_path = 'train\img-test'

In [ ]:
import os
import xml.etree.ElementTree as ET

def select_files_in_folder(path):
    for filename in os.listdir(path):
        fullname = os.path.join(path, filename)
        try:
            tree = ET.parse(fullname)
        except ET.ParseError:
            print(f"Error parsing {fullname}")

select_files_in_folder("train\Annotations Boxes")

In [ ]:
annot_file_list = sorted([os.path.join(annot_data_path, i) for i in os.listdir(annot_data_path) if '.xml' in i])
annot_file_list[:5], annot_file_list[-5:], len(annot_file_list)

In [ ]:
train_file_list = sorted([os.path.join(train_data_path, i) for i in os.listdir(train_data_path) if '.jpg' in i])
train_file_list[:5], train_file_list[-5:], len(train_file_list)

In [ ]:
test_file_list = sorted([os.path.join(test_data_path, i) for i in os.listdir(test_data_path) if '.jpg' in i])
test_file_list[:5], test_file_list[-5:], len(test_file_list)

In [ ]:
image_data_file_list = np.concatenate((train_file_list, test_file_list))
image_data_file_list[:5], image_data_file_list[-5:], len(image_data_file_list)

In [ ]:
# Extracting data from annotation files
meta_list = [] # To store general info for every image
object_list_train = [] # To store object classes info of train dataset
object_list_test = [] # To store object classes info of test dataset

for file in tqdm(annot_file_list):
    meta_dict = {}
    root = ElementTree.parse(file).getroot()
    
    # Filename - extracted
    for path in image_data_file_list:
        if root.find('filename').text in path:
            meta_dict['filename'] = path
            meta_dict['split_type'] = path.split("\\")[1]
    
    # Width - extracted
    meta_dict['width'] = int(root.find('size').find('width').text)
    
    # Height - extracted
    meta_dict['height'] = int(root.find('size').find('height').text)
    
    # Objects - extracted and combined into a single string
    meta_dict['objects'] = ', '.join(np.unique([obj.find('name').text for obj in root.findall('object')]))
    meta_list.append(meta_dict)
    
    # Collecting all the object classes instance and counting total appearance
    for obj in root.findall('object'):
        if meta_dict['split_type'] == 'img-train':
            object_list_train.append(obj.find('name').text)
        elif meta_dict['split_type'] == 'img-test':
            object_list_test.append(obj.find('name').text)
    
# Counting the instance for every object class
object_instance_list_train = Counter(sorted(object_list_train))
object_instance_list_test = Counter(sorted(object_list_test))
    
# Collecting Class list and indexing it also in a sequence
class_dict = {k: v for v, k in enumerate(sorted(np.unique(object_list_train)))}
meta_list[:5], object_instance_list_train, object_instance_list_test, class_dict

In [ ]:
# rast_class_dict = class_dict[1:]
# rast_class_dict

In [ ]:
meta_df = pd.DataFrame(meta_list)
meta_df

In [ ]:
print(meta_df[meta_df['objects'] == 'roundabout_ahead'])

In [ ]:
print(f'Size of the images width: {meta_df.width.unique()[0]} and height: {meta_df.height.unique()[0]}')
print(f'Total number of classes with all possible combination: {len(meta_df.objects.unique())}')
print(f'Total length of the training/validation dataset: {len(meta_df[meta_df["split_type"] == "img-train"])} and testing dataset: {len(meta_df[meta_df["split_type"] == "img-test"])}')

In [ ]:
meta_df.objects.value_counts()[:27].plot(kind='barh').invert_yaxis()
plt.xlabel('Images (Count)')
plt.title('Top 27 Objects Classes');

In [ ]:
plt.figure(figsize=(25, 8))
plt.subplot(1, 2, 1)
meta_df[meta_df["split_type"] == "img-train"].objects.value_counts()[:27].plot(kind='barh').invert_yaxis()
plt.xlabel('Images (Count)')
plt.title('Train/Val Dataset', fontsize=16)
plt.subplot(1, 2, 2)
meta_df[meta_df["split_type"] == "img-test"].objects.value_counts()[:27].plot(kind='barh').invert_yaxis()
plt.xlabel('Images (Count)')
plt.title('Test Dataset', fontsize=16)
plt.suptitle('Top 27 Objects Classes in the Dataset', fontsize=10, fontweight='bold');

In [ ]:
plt.figure(figsize=(28, 8))
plt.subplot(1, 2, 1)
plt.barh(list(object_instance_list_train.keys()), list(object_instance_list_train.values()))
plt.xlabel('Objects (Count)')
plt.title('Train/Val Dataset', fontsize=16)
plt.subplot(1, 2, 2)
plt.barh(list(object_instance_list_test.keys()), list(object_instance_list_test.values()))
plt.xlabel('Objects (Count)')
plt.title('Test Dataset', fontsize=16)
plt.suptitle('Total Count of Object Instances Per Class in the Dataset', fontsize=20, fontweight='bold');

In [ ]:
viz_class = random.sample(meta_df.objects.tolist(), 1)[0]
viz_list = meta_df[meta_df['objects'] == viz_class].filename.tolist()
plt.figure(figsize=(20, 5))
rand = random.sample(viz_list, 4)
for i in range(4):
    plt.subplot(1, 4, i+1)
    plt.imshow(plt.imread(rand[i]))
    plt.suptitle(f'Objects in the Image: {viz_class}', fontsize=20, fontweight='bold')
    plt.axis(False)

In [ ]:
# Creating a function for extracting data
def extract_data_from_xml(xml_file: str):
    """
    A function to extract data like filename, size, classes and bboxes from xml file.
    
    Parameters: xml_file: str, A string containing the path to the file.
    
    Returns: data_dict: dict, A dict containing all the extracted data.
    """
    root = ElementTree.parse(xml_file).getroot()
    
    # Creating dict and list to store data
    data_dict = {}
    data_dict['bboxes'] = []
    
    # Reading the xml file
    for element in root:
        # Getting the filename
        if element.tag == 'filename':
            data_dict['filename'] = element.text
        
        # Getting the image size
        elif element.tag == 'size':
            image_size = []
            for size_element in element:
                image_size.append(int(size_element.text))
            data_dict['image_size'] = image_size
        
        # Getting the bounding box
        elif element.tag == 'object':
            bbox = {}
            for obj_element in element:
                # Object or Class name
                if obj_element.tag == 'name':
                    bbox['class'] = obj_element.text
                # Object bounding box 
                elif obj_element.tag == 'bndbox':
                    for bbox_element in obj_element:
                        bbox[bbox_element.tag] = int(bbox_element.text)
            data_dict['bboxes'].append(bbox)
    return data_dict

In [ ]:
example = extract_data_from_xml(annot_file_list[2])
example

In [ ]:

def convert_dict_to_yolo(data_dict: dict):
    """
    A function to convert the extracted data dict into a text file as per the YOLO format.
    The final text file is saved in the directory "dior_data/yolo_annotations/data_dict['filename'].txt".
    
    Parameters: data_dict: dict, A dict containing the data.
    """
    data = []
    
   
    for bbox in data_dict['bboxes']:
        try:
            class_id = class_dict[bbox['class']]
        except KeyError:
            print(f'Invalid Class. Object class: "{bbox["class"]}" not present in the class list.')
            
        
        img_w, img_h, _ = data_dict['image_size'] 
        
        x_center = ((bbox['xmin'] + bbox['xmax']) / 2) / img_w
        y_center = ((bbox['ymin'] + bbox['ymax']) / 2) / img_h
        width = (bbox['xmax'] - bbox['xmin']) / img_w 
        height = (bbox['ymax'] - bbox['ymin']) / img_h
        
        
        data.append(f'{class_id} {x_center:.3f} {y_center:.3f} {width:.3f} {height:.3f}')
        
    
    yolo_annot_dir = os.path.join('train', 'yolo_annotations')
    if not os.path.exists(yolo_annot_dir):
        os.makedirs(yolo_annot_dir)
    save_file_name = os.path.join(yolo_annot_dir, data_dict['filename'].replace('jpg','.txt'))
    
    
    f = open(save_file_name, 'w+')
    f.write('\n'.join(data))
    f.close()

In [ ]:
class_dict

In [ ]:
print('[INFO] Annotation extraction and creation into Yolo has started.')
for annot_file in tqdm(annot_file_list):
    data_dict = extract_data_from_xml(annot_file)
    convert_dict_to_yolo(data_dict)
print('[INFO] All the annotation are converted into Yolo format.')

In [ ]:
def add_extension_to_files(path, extension):
    for filename in os.listdir(path):
        if not filename.endswith(extension):
            os.rename(os.path.join(path, filename), os.path.join(path, filename + extension))

In [ ]:
input_folder_path = 'train\yolo_annotations'
add_extension_to_files(input_folder_path, '.txt')

In [ ]:
yolo_annot_path = 'train\yolo_annotations'
yolo_annot_file_list = sorted([os.path.join(yolo_annot_path, i) for i in os.listdir(yolo_annot_path) if '.txt' in i])
yolo_annot_file_list[:5], yolo_annot_file_list[-5:], len(yolo_annot_file_list)

In [ ]:
class_dict_idx = dict(zip(class_dict.values(), class_dict.keys()))
class_dict_idx

In [ ]:
def plot_bboxes(img_file: str, annot_file: str, class_dict: dict):
    """
    A function to plot the bounding boxes amd their object classes onto the image.
    
    Parameters:
        img_file: str, A string containing the path to the image file.
        annot_file: str, A string containing the path to the annotation file in yolo format.
        class_dict: dict, A dict containing the classes in the similar sequence as per the annot_file.
    """
    # Reading the image and annot file
    image = cv2.imread(img_file)
    img_h, img_w, _ = image.shape
    
    with open(annot_file, 'r') as f:
        data = f.read().split('\n')
        data = [i.split(' ') for i in data]
        data = [[float(j) for j in i] for i in data]
    
    # Calculating the bbox in Pascal VOC format
    for bbox in data:
        class_idx, x_center, y_center, width, height = bbox
        xmin = int((x_center - width / 2) * img_w)
        ymin = int((y_center - height / 2) * img_h)
        xmax = int((x_center + width / 2) * img_w)
        ymax = int((y_center + height / 2) * img_h)
        
        # Correcting bbox if out of image size
        if xmin < 0:
            xmin = 0
        if ymin < 0:
            ymin = 0
        if xmax > img_w - 1:
            xmax = img_w - 1
        if ymax > img_h - 1:
            ymax = img_h - 1
        
        # Creating the box and label for the image
        cv2.rectangle(image, (xmin, ymin), (xmax, ymax), (255, 255, 0), 2)
        cv2.putText(image, class_dict[class_idx], (xmin, 0 if ymin-10 < 0 else ymin-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 0), 2)
    
    # Displaying the image
    plt.imshow(image)
    plt.axis(False)

In [ ]:
plt.figure(figsize=(25, 8))
rand_int = random.sample(range(len(yolo_annot_file_list)), 3)
for i in range(3):
    plt.subplot(1, 3, i+1)
    plot_bboxes(image_data_file_list[rand_int[i]], yolo_annot_file_list[rand_int[i]], class_dict_idx)

In [ ]:
root_dir = 'datasets'
image_dir = 'datasets/images'
label_dir = 'datasets/labels'
img_train_dir = 'datasets/images/train'
img_val_dir = 'datasets/images/val'
label_train_dir = 'datasets/labels/train'
label_val_dir = 'datasets/labels/val'


total_val_size = int(len(test_file_list) * 0.2)
total_val_size

In [ ]:
if not os.path.exists(img_train_dir):
    os.makedirs(img_train_dir)

for filepath in tqdm(train_file_list):
    if os.path.isfile(filepath):
        shutil.move(filepath, img_train_dir)

In [ ]:
if not os.path.exists(img_val_dir):
    os.makedirs(img_val_dir)

for filepath in tqdm(test_file_list[:total_val_size]):
    if os.path.isfile(filepath):
        shutil.move(filepath, img_val_dir)

In [ ]:
if not os.path.exists(label_train_dir):
    os.makedirs(label_train_dir)

for filepath in tqdm(train_file_list):
    file_path = os.path.join('train/yolo_annotations', filepath.replace('jpg', 'txt').split('\\')[-1])
    if os.path.isfile(file_path):
        shutil.move(file_path, label_train_dir)

In [ ]:
if not os.path.exists(label_val_dir):
    os.makedirs(label_val_dir)

for filepath in tqdm(test_file_list[:total_val_size]):
    file_path = os.path.join('train/yolo_annotations', filepath.replace('jpg', 'txt').split('\\')[-1])
    if os.path.isfile(file_path):
        shutil.move(file_path, label_val_dir)

In [ ]:
from ultralytics import YOLO

In [ ]:
model = YOLO('yolov8n.yaml')

In [ ]:
%%writefile road.yaml
# Path to my dataset
path: 'C:/Users/COMPUTER-JOKAR/Soil Mechanics Laboratory/Mechanics Laboratory/datasets'
train: 'images/train'
val: 'images/val'

# Classes as per mentioned in the annotation
names:
    0: bridge
    1: concrete
    2: guardrail

In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [ ]:
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.Resize((100, 100)), 
    transforms.ToTensor()
])

In [ ]:
results = model.train(data='road.yaml', epochs=50, batch=15, name='yolov8n_epochs50_batch16')

In [ ]:
img_test_dir = 'datasets/images/test'

# Moving the Testing images[80% of test dataset]
if not os.path.exists(img_test_dir):
    os.makedirs(img_test_dir)

test_list = os.listdir('train/img-test')
for filename in tqdm(test_list):
    filepath = os.path.join('train/img-test', filename)
    if os.path.isfile(filepath):
        shutil.move(filepath, img_test_dir)

In [ ]:
label_test_dir = 'datasets/labels/test'

# Moving the testing annotation [80% of testing dataset]
if not os.path.exists(label_test_dir):
    os.makedirs(label_test_dir)

for filename in tqdm(test_list):
    filepath = os.path.join('train/yolo_annotations', filename).replace('jpg', 'txt')
    if os.path.isfile(filepath):
        shutil.move(filepath, label_test_dir)

In [ ]:
%%writefile test_road.yaml
# Path to my dataset
path: 'C:/Users/COMPUTER-JOKAR/Soil Mechanics Laboratory/Mechanics Laboratory/datasets'
train: 
val: 'images/test'

# Classes as per mentioned in the annotation
names:
    0: bridge
    1: concrete
    2: guardrail

In [ ]:
my_model = YOLO('runs/detect/yolov8n_epochs100_batch16/weights/best.pt')

In [ ]:
results = my_model.predict(source='/Users/COMPUTER-JOKAR/Soil Mechanics Laboratory/Mechanics Laboratory/sign_data/train/', save=True, save_txt=True)

In [ ]:
test_results = model.val(data='test_road.yaml', imgsz=800, name='yolov8n_val_on_test')

In [ ]:
Image('runs\detect\yolov8n_val_on_test5\confusion_matrix.png')

In [ ]:
Image('runs\detect\yolov8n_val_on_test5\F1_curve.png')

In [ ]:
Image('runs\detect\yolov8n_val_on_test5\PR_curve.png')

In [ ]:
val_result_image = ['runs/detect/yolov8n_val_on_test5/val_batch0_labels.jpg', 'runs/detect/yolov8n_val_on_test5/val_batch0_pred.jpg', 'runs/detect/yolov8n_val_on_test5/val_batch1_labels.jpg', 'runs/detect/yolov8n_val_on_test5/val_batch1_pred.jpg', 'runs/detect/yolov8n_val_on_test5/val_batch2_labels.jpg', 'runs/detect/yolov8n_val_on_test5/val_batch2_pred.jpg']
plt.figure(figsize=(20, 30))
for i in range(len(val_result_image)):
    plt.subplot(3, 2, i+1)
    plt.imshow(plt.imread(val_result_image[i]))
    plt.axis(False)

In [ ]:
model = YOLO('runs/detect/yolov8n_epochs50_batch164/weights/best.pt')

In [ ]:
img_test_dir = 'datasets/images/test'
label_test_dir = 'datasets/labels/test'
rand_img = random.sample(os.listdir(img_test_dir), 1)[0]
rand_img_path = os.path.join(img_test_dir, rand_img)
rand_label_path = os.path.join(label_test_dir, rand_img).replace('jpg', 'txt')

# Predicting the object using the yolo model
pred_list = model.predict(source=rand_img_path, imgsz=800, save=True, conf=0.5)
pred_img_path = os.path.join('runs\detect\predict', rand_img) # Predict path can change

# Ploting a the true and predicted images with bounding boxes
plt.figure(figsize=(12, 7))
plt.subplot(1, 2, 1)
plot_bboxes(rand_img_path, rand_label_path, class_dict_idx)
plt.title('Original Image')
plt.subplot(1, 2, 2)
plt.imshow(plt.imread(pred_img_path))
plt.title('Predicted Image')
plt.axis(False);

In [ ]:
results = model.predict(source='datasets/images/test', save=True, save_txt=True)

In [ ]:
video_list = ['test_data/bridge_test.mp4', 'test_data/concrete_test.mp4']
for file in video_list:
    pred_list = model.predict(source=file, imgsz=800, save=True, conf=0.5)

In [ ]:
Video('runs/detect/predict2/bridge_test.mp4', width=500, embed=True)

In [ ]:
Video('runs/detect/predict2/concrete_test.mp4', width=500, embed=True)